In [1]:
import keras
keras.__version__

'2.12.0'

# Pierwszy przykład sieci neuronowej

Nie przejmuj się, jeżeli nie zrozumiesz wszystkich elementów tego przykładu. Jest to normalne, jeżeli nie masz doświadczenia w pracy z biblioteką Keras ani innym podobnym do niej pakietem. Prawdopodobnie jeszcze nawet nie zainstalowałeś pakietu Keras… To niczemu nie przeszkadza. W kolejnym rozdziale opiszę każdy element tego przykładu w sposób szczegółowy. W związku z tym nie przejmuj się, jeżeli niektóre rzeczy wydadzą Ci się czarną magią! Musimy od czegoś zacząć.

W zaprezentowanym przykładzie próbujemy rozwiązać problem klasyfikacji obrazów w skali szarości przedstawiających ręcznie zapisane cyfry (obrazy te mają rozdzielczość 28x28 pikseli). Chcemy podzielić je na 10 kategorii (cyfry od 0 do 9). Będziemy korzystać ze zbioru danych MNIST, który jest uznawany przez środowisko analityków za zbiór klasyczny. Istnieje on tak długo, jak długa jest historia uczenia maszynowego. Zbiór ten zawiera 60 000 obrazów treningowych oraz 10 000 obrazów testowych. Został on utworzony przez Narodowy Instytut Standaryzacji i Technologii (NIST) w latach 80. ubiegłego wieku. Rozwiązanie wspomnianego problemu można porównać do wyświetlenia napisu „Witaj, świecie!” podczas nauki nowego języka programowania. Zbiór ten jest również używany w celu sprawdzania tego, czy algorytm działa poprawnie. Jeżeli zaczniesz zawodowo zajmować się uczeniem maszynowym, to odkryjesz, że zbiór MNIST pojawia się ciągle w różnych pracach naukowych, artykułach publikowanych w internecie itd. Na rysunku 2.1 przedstawiono wybrane elementy tego zbioru.




```
# Sformatowano jako kod
```

Zbiór danych MNIST jest dołączony do pakietu Keras w formie czterech tablic Numpy:

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Tablice train_images i train_labels tworzą treningowy zbiór danych. Będzie on używany podczas trenowania modelu. Do testowania posłuży nam testowy zbiór danych, składający się z tablic test_images i test_labels. Obrazy są zakodowane w formie tablic Numpy, a etykiety mają formę tablicy cyfr (od 0 do 9). Do każdego obrazu przypisana jest tylko jedna etykieta.

Przyjrzyjmy się treningowemu zbiorowi danych:

In [3]:
train_images.shape

(60000, 28, 28)

In [ ]:
from google.colab.patches import cv2_imshow

In [4]:
train_images[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [ ]:

cv2_imshow(train_images[0])

In [5]:
len(train_labels)

60000

In [6]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [7]:
train_labels[0]

5

A teraz zobaczmy, jak wyglądają dane testowe:

In [8]:
test_images.shape

(10000, 28, 28)

In [ ]:
cv2_imshow(test_images[0])

In [ ]:
len(test_labels)

10000

In [ ]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Będziemy pracować według następującego przepływu roboczego: najpierw będziemy trenować sieć neuronową na danych treningowych: train_images i train_labels. Sieć nauczy się kojarzyć obrazy i etykiety. Następnie nasza sieć wygeneruje przewidywania dotyczące zbioru test_images, a uzyskane wyniki porównamy z etykietami test_labels.

Zbudujmy naszą sieć. Przypominam, że nie musisz jeszcze rozumieć wszystkiego, co się dzieje w tym przykładzie.

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(256, activation='tanh'))
network.add(layers.Dense(10, activation='softmax'))

Głównym blokiem składowym sieci neuronowej jest warstwa (ang. layer). Jest to moduł przetwarzania danych, który można traktować jako filtr danych. Dane wychodzące z filtra mają bardziej przydatną formę od danych do niego wchodzących. Niektóre warstwy dokonują ekstrakcji reprezentacji kierowanych do nich danych — reprezentacje te powinny ułatwiać rozwiązanie problemu, z którym się zmagamy. Większość uczenia głębokiego składa się z łączenia ze sobą prostych warstw w celu zaimplementowania progresywnej destylacji danych. Model uczenia głębokiego jest jak sito przetwarzające dane składające się z coraz drobniejszych siatek — warstw.

Nasza sieć składa się z sekwencji dwóch warstw Dense, które są ze sobą połączone w sposób gęsty (dochodzi tu do gęstego połączenia). Druga warstwa jest dziesięcioelementową warstwą softmax — warstwa ta zwróci tablicę 10 wartości prawdopodobieństwa (suma wszystkich tych wartości jest równa 1). Każdy z tych wyników określa prawdopodobieństwo tego, że na danym obrazie przedstawiono daną cyfrę (obraz może przedstawiać jedną z dziesięciu cyfr).

Na etapie kompilacji musimy określić jeszcze trzy rzeczy w celu przygotowania sieci do trenowania. Są to:

* Funkcja straty — funkcja ta definiuje sposób pomiaru wydajności sieci podczas przetwarzania treningowego zbioru danych, a więc pozwala na dostrajanie parametrów sieci we właściwym kierunku.
* Optymalizator — mechanizm dostrajania sieci na podstawie danych zwracanych przez funkcje straty.
* Metryki monitorowane podczas trenowania i testowania — tutaj interesuje nas jedynie dokładność (część obrazów, która została właściwie sklasyfikowana).

W kolejnych dwóch rozdziałach wyjaśnię cel stosowania funkcji straty i optymalizatora.

In [ ]:
network.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


Zanim rozpoczniemy trenowanie, zmienimy kształt danych tak, aby przyjęły kształt oczekiwany przez sieć, i przeskalujemy je do wartości z zakresu [0, 1]. Początkowo nasze obrazy treningowe były zapisywane w postaci macierzy o wymiarach (60000, 28, 28), zawierającej wartości z zakresu [0, 255], i typie uint8. Przekształcamy je w tablicę typu float32 o wymiarach (60000, 28 * 28), zawierającą wartości od 0 do 1.

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [ ]:
train_images[0]
train_images.shape

(60000, 784)

In [ ]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Musimy dodatkowo zakodować etykiety za pomocą kategorii

> Blok z wcięciem



In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
train_labels[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
network.fit(train_images, train_labels, epochs=10, batch_size=128)

Epoch 1/10
469/469 [==============================] - 8s 3ms/step - loss: 0.2162 - accuracy: 0.9351
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0784 - accuracy: 0.9764
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0509 - accuracy: 0.9837
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0344 - accuracy: 0.9888
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0269 - accuracy: 0.9912
Epoch 6/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0224 - accuracy: 0.9924
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0167 - accuracy: 0.9943
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0125 - accuracy: 0.9959
Epoch 9/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0133 - accuracy: 0.9955
Epoch 10/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0146 - accuracy: 0.9950

Podczas trenowania wyświetlane są dwie wartości: strata sieci jej dokładność (obie wartości dotyczą treningowego zbioru danych).

Podczas trenowania szybko osiągamy dokładność 0,989 (98,9%). Teraz możemy sprawdzić dokładność przetwarzania testowego zbioru danych:


In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0768 - accuracy: 0.9815


In [ ]:
print('test_acc:', test_acc)

test_acc: 0.9815000295639038


In [ ]:
testowe = test_images[0].reshape((1, 28*28))
import numpy as np
print(np.argmax(network.predict(testowe)))
print(np.max(network.predict(testowe)))
network.predict(testowe)

1/1 [==============================] - 0s 68ms/step
7
1/1 [==============================] - 0s 17ms/step
1.0
1/1 [==============================] - 0s 19ms/step


array([[1.1388167e-11, 4.5555106e-09, 1.1332519e-08, 3.5151266e-09,
        1.4076141e-11, 7.1241618e-10, 3.1857131e-15, 1.0000000e+00,
        3.7131347e-11, 1.5454864e-08]], dtype=float32)


W przypadku testowego zbioru danych uzyskaliśmy dokładność na poziomie 97,8%, a więc wartość nieco niższą niż dla zbioru treningowego. Różnica między tymi wartościami wynika z nadmiernego dopasowania. Modele uczenia maszynowego mają tendencję do niższej dokładności przetwarzania nowych danych, niż to miało miejsce w przypadku danych treningowych. Zagadnienie to jest głównym tematem rozdziału 3.

To tyle, jeżeli chodzi o nasz pierwszy przykład — właśnie zobaczyłeś, że zbudowanie i wytrenowanie sieci neuronowej klasyfikującej zapis ręczny cyfr może zająć mniej niż 20 linii kodu Pythona. W kolejnym rozdziale opiszę szczegółowo wszystkie linie tego kodu i wyjaśnię, jakie operacje są wykonywane w tle. Dowiesz się, czym są tensory, obiekty przeznaczone do przechowywania danych sieci i operacje na tensorach. Poznasz budowę warstw sieci i algorytm spadku gradientowego, który umożliwia sieci uczenie się na podstawie treningowego zbioru danych.
